In [107]:
import pandas as pd  
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import json
import urllib

In [108]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output, State, MATCH, ALL
import dash_table
from dash.exceptions import PreventUpdate

import plotly.graph_objects as go
import dash_bootstrap_components as dbc

In [109]:
####################
#DASH visualization
####################


In [110]:
####################################
#These are the controls for the model
####################################

LEFT_COLUMN = dbc.Jumbotron(   html.Div([
    html.P('(Select the parameters for the Market Model)', style={"fontSize": 16, "font-weight": "lighter"}),
    #Slider 1
    html.Label("Select % of Patients INELIGIBLE for Treatment", className="lead"),
    dcc.Slider(id = 'tx_inelgible', 
               min = 0,
               max = 1,
               value = .25,
               step=0.05,
               marks = {0: '0%',
                        .05: '5%',
                        .1:'10%',
                        .15:' ',
                       .2:'20%',
                       .25:' ',
                       .3:'30%',
                       .35:' ',
                       .4:'40%',
                       .45:' ',
                       .5:'50%',
                       .55:' ',
                       .6:'60%',
                       .65:' ',
                       .7:'70%',
                       .75:' ',
                       .8:'80%',
                       .85:' ',
                       .9:'90%',
                       .95:' ',
                       1:'100%'},
                        
               included = True),
    html.Label("Select the Peak Penetrance of the Product", className="lead"),
    #Slider 2
    dcc.Slider(id = 'penetrance_num', 
               min = 0,
               max = 1,
               value = .25,
               marks = {0: '0%',
                        .05: '5%',
                        .1:'10%',
                        .15:' ',
                       .2:'20%',
                       .25:' ',
                       .3:'30%',
                       .35:' ',
                       .4:'40%',
                       .45:' ',
                       .5:'50%',
                       .55:' ',
                       .6:'60%',
                       .65:' ',
                       .7:'70%',
                       .75:' ',
                       .8:'80%',
                       .85:' ',
                       .9:'90%',
                       .95:' ',
                       1:'100%'},
               step=0.05,
               included = True),
                
    html.Label("Select the Duration of Treatment, Months", className="lead"),
    #Slider 3
    dcc.Slider(id = 'months_treatment_num', 
               min = 0,
               max = 12,
               marks = {i: '{}'.format(i) for i in range(12)},
               value = 9,
               step=1),
    
    html.Label("Monthly Cost of Therapy, $", className="lead"),
    #Slider 4
    dcc.Slider(id = 'price', 
               min = 0,
               max = 50000,
               marks = {100:'$100',
                        1000:'$1000', 
                        5000:'$5000',
                        10000:'$10,000',
                        25000:'$25,000',
                        50000:'$50,000'
                        
                       },
               value = 8000,
               step=100),
    
    html.Label("Probability of Success", className="lead"),
    #Slider 5
    dcc.Slider(id = 'POS', 
               min = 0,
               max = 1,
               marks = {0: '0%',
                        .05: '5%',
                        .1:'10%',
                        .15:' ',
                       .2:'20%',
                       .25:' ',
                       .3:'30%',
                       .35:' ',
                       .4:'40%',
                       .45:' ',
                       .5:'50%',
                       .55:' ',
                       .6:'60%',
                       .65:' ',
                       .7:'70%',
                       .75:' ',
                       .8:'80%',
                       .85:' ',
                       .9:'90%',
                       .95:' ',
                       1:'100%'},
               value = .4,
               step=.05),
    #Slider 6
    html.Label("Gross to Net i.e. 0 = all the money goes back to the developer", className="lead"),
    dcc.Slider(id = 'gross_net', 
               min = 0,
               max = 1,
                marks = {0: '0%',
                        .05: '5%',
                        .1:'10%',
                        .15:' ',
                       .2:'20%',
                       .25:' ',
                       .3:'30%',
                       .35:' ',
                       .4:'40%',
                       .45:' ',
                       .5:'50%',
                       .55:' ',
                       .6:'60%',
                       .65:' ',
                       .7:'70%',
                       .75:' ',
                       .8:'80%',
                       .85:' ',
                       .9:'90%',
                       .95:' ',
                       1:'100%'},
               value = .75,
               step=.05),
    #Slider 7
    html.Label("Discount Rate", className="lead"),
    dcc.Slider(id = 'disc_rate', 
               min = 0,
               max = 1,
                marks = {0: '0%',
                        .05: '5%',
                        .1:'10%',
                        .15:' ',
                       .2:'20%',
                       .25:' ',
                       .3:'30%',
                       .35:' ',
                       .4:'40%',
                       .45:' ',
                       .5:'50%',
                       .55:' ',
                       .6:'60%',
                       .65:' ',
                       .7:'70%',
                       .75:' ',
                       .8:'80%',
                       .85:' ',
                       .9:'90%',
                       .95:' ',
                       1:'100%'},
               value = .1,
               step=.05),
    html.Button('Submit', id='save_model', n_clicks = 0),
    html.Div(id='button_text',
             children='Enter a value and press submit'),
   
        ],  style={'width': '85%', 'height':'auto','display': 'inline-block'}),)


In [111]:
HEADERS = dbc.CardColumns([
    dbc.Card(dbc.CardBody(html.H4(html.Div(id = 'price_num')))),
    dbc.Card(dbc.CardBody(html.H4(html.Div(id = 'price_month_num')))),
    dbc.Card(dbc.CardBody(html.H4(html.Div(id="entry-year")))),
    dbc.Card(dbc.CardBody(html.H4(html.Div(id = 'tx_inelgible_num')))),
    dbc.Card(dbc.CardBody(html.H4(html.Div(id = 'POS_num')))),
    dbc.Card(dbc.CardBody(html.H4(html.Div(id = 'NPV')))),  
    dbc.Card(dbc.CardBody(html.H4(html.Div(id = 'gross_net_num')))),    
    dbc.Card(dbc.CardBody(html.H4(html.Div(id = 'disc_rate_num'))))])

In [112]:
REVENUE_GRAPHS =   dbc.CardBody([dbc.CardGroup([dbc.Card(dcc.Loading(dcc.Graph(id = "population-graph"))),
                                               dbc.Card(dcc.Loading(dcc.Graph(id = "revenue-graph")))]),
                               #dbc.CardGroup([dbc.Card(dcc.Loading(dcc.Graph(id = "all_graph")))])
                                ])
                                          

In [113]:

BODY = dbc.Container(
    [dbc.Row([HEADERS]),
     dbc.Row([dbc.Col(LEFT_COLUMN, md=4, align="top"),
             dbc.Col(REVENUE_GRAPHS, md = 8, align = "top"),
             dbc.Col(html.Div(children = [html.Div(id = 'tableDiv', children = dash_table.DataTable( 
                             id='table',
             export_format='xlsx',
    export_headers='display',
    merge_duplicate_headers=True
))]))],
            style={"marginTop": 20}),
    ], fluid = True)

In [114]:
###############
#Tab 2 layout
###############

NPV_Ranking = html.Div(id = 'npv_div', children = [dash_table.DataTable( 
                             id='npv_table',
        style_cell={'padding': '5px'},

        style_header={
        'backgroundColor': 'white',
        'fontWeight': 'bold'
        },
             export_format='xlsx',
    export_headers='display',
    merge_duplicate_headers=True
)])

In [115]:
#################################
#APP- putting the pieces together
#################################

app = dash.Dash(__name__,external_stylesheets=[dbc.themes.SOLAR])
app.layout = html.Div([
    #Tab 1 - market model
    dcc.Tabs([
    dcc.Tab(label='Market Model Builder', children=[html.Div([
    dcc.Store(id='memory_output'), 
    dcc.Store(id = 'dataframe_list', data = []),
    html.Label("Enter the year the product enters the market"),
    dcc.Input(id = 'entry_year_num', type="number",
            debounce=True, placeholder=int(2022)),
    html.Label("How many years of patent life?"),
    dcc.Input(id = 'fwd_num', type="number",
            debounce=True, placeholder=int(5)),
    html.Label("What's the disease?"),
    dcc.Input(id = 'dz_str', type="text",
            debounce=True, placeholder='Lupus'),
    html.Label("What's the drug name?"),
    dcc.Input(id = 'drug_str', type="text",
            debounce=True, placeholder='Aspirin'),
    html.Label("What's the incidence of the disease"),
    dcc.Input(id = 'incidence_per', type="number",
            debounce=True, placeholder='0.0001'),
    html.Button('CREATE MODEL', id='create_model'),

    html.Div([BODY]),
        
    ])]),
    #Tab 2 - analysis
    dcc.Tab(label='Analysis', children=[html.Div(children = [NPV_Ranking]
    )]
    )
    ])

])


In [116]:
####################################
#Callbacks - Drug name
####################################
@app.callback(
    Output("drug_str_name", "children"),
    [Input("drug_str", "value")])
def update_output(drug_str):
    return 'Drug Name: {}'.format(drug_str)

In [117]:
####################################
#Callbacks - treatment ineligible %
####################################
@app.callback(
    Output("entry-year", "children"),
    [Input("entry_year_num", "value")])
def update_output(entry_year_num):
    return 'Entry Year: {}'.format(entry_year_num)


In [118]:
########################
#Callbacks - Entry Year
########################
@app.callback(
    Output("tx_inelgible_num", "children"),
    [Input("tx_inelgible", "value")])
def update_output(tx_inelgible):
    return '% Treatment Eligible: {}%'.format((1-tx_inelgible)*100)


In [119]:
####################################
#Callbacks - Annual Price
####################################
@app.callback(
    Output("price_num", "children"),
    [Input("price", "value")])
def update_output(price):
    return '$ {} per Year'.format(price*12)

In [120]:
####################################
#Callbacks - Monthly Price
####################################
@app.callback(
    Output("price_month_num", "children"),
    [Input("price", "value")])
def update_output(price):
    return '$ {} per Month'.format(price)

In [121]:
####################################
#Callbacks - Probability of Success
####################################
@app.callback(
    Output("POS_num", "children"),
    [Input("POS", "value")])
def update_output(POS):
    return 'Probability of Success: {} %'.format(POS*100)

In [122]:
####################################
#Callbacks - Gross to Net
####################################
@app.callback(
    Output("gross_net_num", "children"),
    [Input("gross_net", "value")])
def update_output(gross_net):
    return 'Gross-to-net: {} %'.format(gross_net*100)

In [123]:
####################################
#Callbacks - Discount Rate
####################################
@app.callback(
    Output("disc_rate_num", "children"),
    [Input("disc_rate", "value")])
def update_output(disc_rate):
    return 'Discount Rate: {} %'.format(disc_rate*100)

In [124]:
############################################################
#Callbacks - The App
############################################################
@app.callback(
    Output('memory_output', 'data'),
    [Input('entry_year_num', 'value'),
    Input('fwd_num', 'value'),
    Input('dz_str', 'value'),
    Input('incidence_per', 'value'),
    Input('tx_inelgible', 'value'), 
    Input('penetrance_num', 'value'),
    Input('months_treatment_num', 'value'),
    Input('price', 'value'),
    Input('POS', 'value'),
    Input('gross_net', 'value'),
    Input('disc_rate', 'value'),
    Input('create_model', 'n_clicks')])
def market_model(entry_year_num, fwd_num, dz_str, incidence_per, tx_inelgible, penetrance_num, months_treatment_num, 
                 price, POS, gross_net, disc_rate, n_clicks):
    if n_clicks is None:
        raise PreventUpdate
    else:
        ####################
        #Parse US population
        ####################
        url = 'https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=us:*'
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'lxml')
        uspop_data = json.loads(soup.text)
        us_raw = pd.DataFrame(uspop_data[1:], columns=uspop_data[0])
        us_raw['POP']
        url = 'http://www.worldometers.info/world-population/us-population/'
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'lxml')
        tables = soup.select('table')
        us_pop_tbl = tables[1]
        us_pop_headers = us_pop_tbl.find_all('th')
        #Columns
        us_pop_columns = []
        for x in us_pop_headers:
            us_pop_columns.append(x.text)

        us_pop_rows = []
        for row in us_pop_tbl.find_all('tr'):
            us_pop_txt = [x.text for x in row.find_all('td')]
            us_pop_rows.append(us_pop_txt)

        #Now put them into a dataframe
        us_pop_df = pd.DataFrame(us_pop_rows[1:]) #first row is blank, so get rid of it
        us_pop_df.columns = us_pop_columns   
        #############################################
        #Start asking for epidemiology of the disease
        #############################################
        us_mkt = us_pop_df[['Population', 'Yearly %  Change', 'Yearly Change']].loc[0:3]
        us_mkt = us_mkt[::-1]
        last_year = int(us_pop_df['Year'].loc[0])
        entry_year = int(entry_year_num)

        fwd = entry_year +fwd_num - last_year
        years = pd.DataFrame({'Year':range(last_year-3, last_year+fwd)})
        temp = pd.concat([us_mkt, years], axis = 1)
        #Need to convert all the strings to int
        us_mkt_2 = temp[['Year', 'Population', 'Yearly %  Change', 'Yearly Change']]
        us_mkt_2['Population'] = us_mkt_2['Population'].str.replace(',', '')
        us_mkt_2['Yearly %  Change'] = us_mkt_2['Yearly %  Change'].str.replace('%','')
        us_mkt_2['Yearly %  Change'] = us_mkt_2['Yearly %  Change'].str.replace(' ','')
        us_mkt_2['Yearly Change'] = us_mkt_2['Yearly Change'].str.replace(',', '')
        us_mkt_2 = us_mkt_2.apply(pd.to_numeric)
        #Multiply by .01 to for appropriate representation of % for Yearly % Chagne
        us_mkt_2['Yearly %  Change'] = us_mkt_2['Yearly %  Change']*.01 
        us_mkt_2['Yearly %  Change'] = us_mkt_2['Yearly %  Change'].fillna(method = 'ffill')
        #########################
        #US Population
        #########################
        us_mkt_2 = us_mkt_2.set_index('Year')
        for index, row in temp.iterrows():
            us_mkt_2['Population'].loc[last_year:] = us_mkt_2['Population'].shift(1)*(1-us_mkt_2['Yearly %  Change']) 
            us_mkt_2['Yearly Change'].iloc[1:] = us_mkt_2['Population']- us_mkt_2['Population'].shift(1)
        disease = dz_str
        us_mkt_2['Incidence percentage'] = float(incidence_per)
        for index, row in us_mkt_2.iterrows():
            us_mkt_2['Incidence ' +disease] = us_mkt_2['Population'] * us_mkt_2['Incidence percentage']
        #This is to describe 

        us_mkt_2['Treatment ineligible'] = float(tx_inelgible)
        for index, row in us_mkt_2.iterrows():
            us_mkt_2['# ' + disease+ ' eligible for treatment'] = us_mkt_2['Incidence ' +disease]*(1-us_mkt_2['Treatment ineligible'])

        #########################
        #Market Sizing
        #########################   
        peak = entry_year+fwd
        peak_penetrance = float(penetrance_num)
        us_mkt_2['Market Penetrance'] = 0

        ##################################################################
        #Assume a 5 year growth to peak penetrance from the year of entry 
        ##################################################################
        if len(us_mkt_2.loc[entry_year:])>=5:
            frac= []
            x= 0
            while len(frac)<5:
                x+=1/5
                temp = x*peak_penetrance
                frac.append(temp)
        elif len(us_mkt_2.loc[entry_year:])<5:
            frac= []
            x= 0
            while len(frac)<fwd_num:
                x+=1/5
                temp = x*peak_penetrance
                frac.append(temp)
        else:
            pass
        ######################################################################################################################  
        #Replace the market penetration with the ramp up time frame, then fill foward any additional years with the peak sales
        ######################################################################################################################
        us_mkt_2.loc[entry_year:entry_year+len(frac)-1,'Market Penetrance'] = frac
        us_mkt_2.loc[entry_year+len(frac):,'Market Penetrance'] =np.nan
        us_mkt_2.loc[entry_year+len(frac):,'Market Penetrance'] =us_mkt_2.loc[entry_year+len(frac)-1:,'Market Penetrance'].fillna(method='ffill')

        for index, row in us_mkt_2.iterrows():
            us_mkt_2['Number of patients Treated'] = us_mkt_2['Market Penetrance']*us_mkt_2['# ' + disease+ ' eligible for treatment']

        ###################################
        #Estimating of treatment duration 
        ###################################    
        us_mkt_2['Duration of Treatment'] = int(months_treatment_num)
        us_mkt_2['Price'] = 0
        us_mkt_2['Price'].loc[entry_year] = int(price)
        us_mkt_2['Price']
        us_mkt_2['Price increase %']= .02
        for index, row in us_mkt_2.iterrows():
            us_mkt_2['Price'].loc[entry_year+1:] = us_mkt_2['Price'].shift(1)*(1+us_mkt_2['Price increase %'])

        us_mkt_2['G/N'] = 0
        us_mkt_2['G/N'].loc[entry_year:] = float(gross_net)
        for index, row in us_mkt_2.iterrows():
            us_mkt_2['net price/month'] = us_mkt_2['Price']*(1-us_mkt_2['G/N'])
            us_mkt_2['Revenue'] = us_mkt_2['net price/month']*us_mkt_2['Duration of Treatment']*us_mkt_2['Number of patients Treated']

        ###########################################
        #Probability of Success
        ###########################################
        us_mkt_2['Probability of Success'] = float(POS)
        for index, row in us_mkt_2.iterrows():
            us_mkt_2['Risk Adjusted Revenue'] = us_mkt_2['Revenue']*us_mkt_2['Probability of Success']

        ###########################################
        #Discount Period and Discount Factor, NPV
        ###########################################

        discount_rate = float(disc_rate)
        periods = []
        for x in range(1, fwd+1):
            period = x
            periods.append(period)
        us_mkt_2['discount period'] =1
        us_mkt_2['discount period'].loc[last_year:] = periods
        for index, row in us_mkt_2.iterrows():
            us_mkt_2['discount factor'] = 1/(1+discount_rate)**us_mkt_2['discount period']
            us_mkt_2['NPV'] =  us_mkt_2['Risk Adjusted Revenue']*us_mkt_2['discount factor']

        us_mkt_2 = us_mkt_2.loc[entry_year-1:]

        return us_mkt_2.to_dict()


In [125]:
##############################################################################
#Call backs for NPV calculation
##############################################################################
@app.callback(
    Output('NPV', 'children'),
    [Input('memory_output', 'data')])
def NPV_calc(data):
    dff = pd.DataFrame(data)
    NPV = sum(dff['NPV'])
    return 'NPV: ${:,.2f}'.format(NPV)

In [126]:
##############################################################################
#Call backs to return a graph of the number of patients eligible for treatment
##############################################################################
@app.callback(
    Output('population-graph', 'figure'),
    [Input('memory_output', 'data')])
def population_graph(data):

    dff = pd.DataFrame(data)
    fig = px.bar(data, 
           x = dff.index,
           y= 'Number of patients Treated')
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    return fig

In [127]:
########################################
#Call backs to return a graph of Revenue
########################################
@app.callback(
    Output('revenue-graph', 'figure'),
    [Input('memory_output', 'data')])
def revenue_graph(data):

    dff = pd.DataFrame(data)
    fig = px.bar(data, 
           x = dff.index,
           y= ['Revenue', 'Risk Adjusted Revenue'])
    fig.update_layout(barmode='group')
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    return fig

In [128]:
################
#Save DataFrame
################
@app.callback(
    Output('dataframe_list', 'data'),
    [Input('save_model', 'n_clicks')],
    [State('memory_output', 'data'),
    State("drug_str", "value"),
    State('dataframe_list', 'data')
    ])
def save_dataframe(n_clicks, memory_output, drug_str, master_list):
    temp = pd.DataFrame(master_list)
    if n_clicks  is None:
        raise PreventUpdate
    df = pd.DataFrame(memory_output)
    for index, row in df.iterrows():
        df.loc[1:, 'drug_name'] = drug_str
    #master_df = temp.append(df, ignore_index=True)
    master_df = pd.concat([temp, df],ignore_index=True)
    master_df = master_df.drop_duplicates()
    master_df = master_df.dropna(subset = ['drug_name'])
    return master_df.to_dict()
    

In [129]:
################
#output Datatable
################
@app.callback(
    [Output('table', 'columns'),
     Output('table', 'data')],
    [Input('dataframe_list','data')])
def create_table(data):
    df = pd.DataFrame(data)
    columns=[{"name": i, "id": i} for i in df.columns]
    return columns, df.to_dict('records')


In [130]:
################
#Tab 2 callbacks
################

In [131]:
############
#NPV Ranking
############
@app.callback(
[Output('npv_table', 'columns'),
Output('npv_table', 'data')],
[Input('dataframe_list','data')])

def npv_ranking(data):
    future_revenue = pd.DataFrame(data)
    future_revenue_sum = future_revenue.groupby('drug_name')['NPV'].sum().reset_index()
    future_revenue_sum = future_revenue_sum.sort_values(by = 'NPV', ascending = False)
    columns=[{"name": i, "id": i} for i in future_revenue_sum.columns]
    
    return columns, future_revenue_sum.to_dict('records')

In [132]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

